## K-Means Clustering: Global Superstore (Week 2)

This notebook performs K-Means clustering on the cleaned sales dataset. It includes:
- Scaling features
- Initial K-Means (k=3)
- Elbow and Silhouette methods to select k
- PCA-based 2D visualization
- Basic cluster interpretation and exports

In [ ]:
# Imports and setup
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

# Plot style
sns.set(style='whitegrid', context='notebook')

DATA_DIR = '/workspace/data/raw'
POSSIBLE_DIRS = [
    os.path.join(DATA_DIR, 'GLOBAL SUPER STORE _SALES DATA ANALYSIS'),
    DATA_DIR
]
OUTPUT_DIR = '/workspace/outputs'
PROCESSED_DIR = '/workspace/data/processed'
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)
print('Environment ready.')

In [ ]:
# Discover dataset files (CSV or Excel)
candidate_files = []
for d in POSSIBLE_DIRS:
    if os.path.isdir(d):
        candidate_files.extend(glob.glob(os.path.join(d, '*.csv')))
        candidate_files.extend(glob.glob(os.path.join(d, '*.xlsx')))
        candidate_files.extend(glob.glob(os.path.join(d, '*.xls')))

print('Found files:', candidate_files)

# Heuristic: prefer Excel if present with 'Global' in name, else first file
selected_path = None
for f in candidate_files:
    if 'Global' in os.path.basename(f) and f.lower().endswith(('.xlsx', '.xls')):
        selected_path = f
        break
if selected_path is None and candidate_files:
    selected_path = candidate_files[0]

print('Selected file:', selected_path)
assert selected_path is not None, 'No data file found. Please place a CSV/XLSX in data/raw.'

In [ ]:
# Load dataset (supports CSV and Excel with sheet detection)
file_ext = os.path.splitext(selected_path)[1].lower()
if file_ext == '.csv':
    df_raw = pd.read_csv(selected_path)
else:
    # Try common sheet names or first sheet
    xl = pd.ExcelFile(selected_path)
    sheet_name = None
    for name in xl.sheet_names:
        if str(name).strip().lower() in {'orders', 'order', 'sales', 'data'}:
            sheet_name = name
            break
    df_raw = pd.read_excel(selected_path, sheet_name=sheet_name)

print(df_raw.shape)
df_raw.head(3)

In [ ]:
# Basic cleaning: keep numeric features, drop id/date-like columns
# You can customize columns as needed for the dataset

# Identify numeric columns
numeric_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()

# Drop obviously non-feature numeric columns if present (heuristic)
cols_to_exclude = set()
for col in numeric_cols:
    if any(key in col.lower() for key in ['id']):
        cols_to_exclude.add(col)

feature_cols = [c for c in numeric_cols if c not in cols_to_exclude]
assert len(feature_cols) > 0, 'No numeric feature columns found.'

print('Feature columns:', feature_cols)
X = df_raw[feature_cols].copy()

# Impute missing values and scale
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

print('Scaled shape:', X_scaled.shape)

In [ ]:
# Step 5: Initial K-Means with k=3
kmeans_initial = KMeans(n_clusters=3, n_init=10, random_state=42)
initial_labels = kmeans_initial.fit_predict(X_scaled)

df = df_raw.copy()
df['Cluster'] = initial_labels

print('Initial inertia:', kmeans_initial.inertia_)
df[['Cluster']].head()

In [ ]:
# Step 6: Determine optimal clusters via Elbow and Silhouette
inertia = []
sil_scores = []
K_RANGE = range(2, 11)
for k in K_RANGE:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = km.fit_predict(X_scaled)
    inertia.append(km.inertia_)
    sil = silhouette_score(X_scaled, labels)
    sil_scores.append(sil)

# Plot Elbow
plt.figure(figsize=(6,4))
plt.plot(list(K_RANGE), inertia, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'elbow.png'), dpi=150)
plt.show()

# Plot Silhouette
plt.figure(figsize=(6,4))
plt.plot(list(K_RANGE), sil_scores, marker='o', color='red')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'silhouette.png'), dpi=150)
plt.show()

best_k = int(K_RANGE[int(np.argmax(sil_scores))])
print('Best k by silhouette:', best_k)

In [ ]:
# Step 6b: Fit KMeans with best k
kmeans_best = KMeans(n_clusters=best_k, n_init=10, random_state=42)
best_labels = kmeans_best.fit_predict(X_scaled)

df['Cluster'] = best_labels
print('Best inertia:', kmeans_best.inertia_)
df['Cluster'].value_counts().sort_index()

In [ ]:
# Step 7: PCA visualization (2 components)
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(7,6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['Cluster'], cmap='viridis', s=18, alpha=0.8)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Customer Segments via PCA')
plt.legend(*scatter.legend_elements(), title='Cluster', loc='best')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'pca_clusters.png'), dpi=150)
plt.show()

print('Explained variance ratio:', pca.explained_variance_ratio_)

In [ ]:
# Simple cluster interpretation: compute feature means by cluster
cluster_profile = df.groupby('Cluster')[feature_cols].mean().round(2)
cluster_counts = df['Cluster'].value_counts().sort_index()

print('Cluster counts:\n', cluster_counts)
cluster_profile

In [ ]:
# Save outputs: clustered dataset and metrics
clustered_path = os.path.join(PROCESSED_DIR, 'clustered_data.csv')
df.to_csv(clustered_path, index=False)

metrics = {
    'best_k': int(best_k),
    'best_inertia': float(kmeans_best.inertia_),
    'silhouette_scores': {int(k): float(s) for k, s in zip(K_RANGE, sil_scores)},
    'elbow_inertia': {int(k): float(v) for k, v in zip(K_RANGE, inertia)}
}
with open(os.path.join(OUTPUT_DIR, 'clustering_metrics.json'), 'w') as f:
    json.dump(metrics, f, indent=2)

print('Saved:', clustered_path)
print('Saved plots to', OUTPUT_DIR)

### Initial interpretation of clusters

Use the table above to infer customer segments (examples):
- Cluster with highest average `Sales` and `Profit` → high-value customers.
- Cluster with high `Discount` but low `Profit` → discount-sensitive, lower profitability.
- Cluster with lower `Sales` but moderate `Quantity` → frequent small orders.

Adjust based on your dataset’s feature names and domain context.